# Analysis and Visualization

This notebook focuses on the model in action and addresses the research question/problem being investigated. It includes plots and visualizations as necessary to address the question and/or research topic.


## Justin


## Vince
